In [3]:
from music21 import converter, instrument, note, chord, midi
import glob

import pandas as pd
import numpy as np
from numpy import newaxis

import random
import itertools

import os
from os import listdir
from os.path import isfile
import math

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.collections
from matplotlib import gridspec

import datetime
from tqdm import *
from datetime import *

import keras
from keras.applications import *
import lightgbm as lgbm
import os
from scipy import ndimage
from scipy import misc
from scipy import signal
import cv2
plt.ion()
plt.show()

# import keras
# from keras.preprocessing.image import ImageDataGenerator
# from keras.layers import Input, Dense, Lambda, Layer,Flatten, Conv2D, MaxPooling2D,UpSampling2D, concatenate, Dropout,Conv2DTranspose, LSTM
# from keras.models import Model, Sequential
# from keras import backend as K
# from keras import metrics
# from keras.models import load_model
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.layers import Activation, Dense
# from music21 import instrument, note, stream, chord
import shutil
import collections
from collections import *
from keras import optimizers
import tensorflow as tf
from IPython.display import clear_output
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

# from sklearn.utils import class_weight
# from skimage.io import imread, imshow, imread_collection, concatenate_images
# from skimage.transform import resize
# from skimage.morphology import label
# from os.path import isfile, join
# from sklearn import preprocessing
# from keras.utils import to_categorical
# from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
# from mpl_toolkits.axes_grid1 import ImageGrid
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"



from skimage import filters
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

In [4]:
notes = []
for file in glob.glob("data/*.mid"):
    print('file', file)
    midiP = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midiP)
    print('parts ', parts)
    if parts: # file has instrument parts
        print('has parts')
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midiP.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
            
len(notes)

file data\a.mid
parts  <music21.stream.Score 0x18d5bb2a2b0>
has parts
file data\Berceuse.mid
parts  <music21.stream.Score 0x18d5d147438>
has parts
file data\Tchaikovsky Lake Of The Swans Act 1 1mov.mid
parts  <music21.stream.Score 0x18d642ed4e0>
has parts
file data\Tchaikovsky Lake Of The Swans Act 1 2mov.mid
parts  <music21.stream.Score 0x18d69119550>
has parts
file data\Tchaikovsky Lake Of The Swans Act 1 3mov.mid
parts  <music21.stream.Score 0x18d5e757390>
has parts
file data\Tchaikovsky Lake Of The Swans Act 1 4mov.mid
parts  <music21.stream.Score 0x18d66cf1eb8>
has parts
file data\Tchaikovsky Lake Of The Swans Act 1 5mov.mid
parts  <music21.stream.Score 0x18d5ee37dd8>
has parts
file data\Tchaikovsky Lake Of The Swans Act 1 6mov.mid
parts  <music21.stream.Score 0x18d62ca6710>
has parts
file data\Tchaikovsky Lake Of The Swans Act 1 7-8movs.mid
parts  <music21.stream.Score 0x18d66ff2c50>
has parts
file data\Tchaikovsky Lake Of The Swans Act 1 9mov.mid
parts  <music21.stream.Score 0x1

8215

In [ ]:
notes

In [12]:
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
n_vocab = len(set(notes))
n_vocab

150

In [8]:
len(pitchnames)
print(pitchnames)

144

['0', '0.1', '0.2.5', '0.2.6', '0.3', '0.3.6', '0.4', '0.4.6', '0.4.7', '0.5', '0.6', '1', '1.3', '1.4', '1.4.7', '1.4.8', '1.6', '1.7', '10', '10.0', '10.1', '10.1.4', '10.2', '10.3', '11', '11.0.4', '11.1', '11.2', '11.2.5', '11.3', '11.3.6', '11.3.6.7', '11.4', '2.4.7', '2.4.8', '2.5.8', '2.5.9', '2.8', '3', '3.5.8', '3.5.9', '3.6', '3.6.9', '3.7', '3.8', '3.9', '4', '4.10', '4.6', '4.6.11', '4.6.7', '4.6.9', '4.7', '4.7.11', '4.8', '4.8.11', '4.9', '5', '5.11', '5.8', '5.9', '6', '6.11', '6.8', '6.9', '6.9.0', '7', '7.0', '7.10', '7.10.0', '7.11', '7.8', '8', '8.0', '8.1', '8.10', '8.11', '8.9', '9', '9.0', '9.0.4', '9.1', '9.11.3', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'B-2', 'B-3', 'B-4', 'B-5', 'B-6', 'B2', 'B3', 'B4', 'B5', 'C#1', 'C#2', 'C#3', 'C#4', 'C#5', 'C#6', 'C#7', 'C3', 'C4', 'C5', 'C6', 'C7', 'D4', 'D5', 'D6', 'E-2', 'E-3', 'E-4', 'E-5', 'E-6', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'F#2', 'F#3', 'F#4', 'F#5', 'F#6', 'F3', 'F4', 'F5', 'F6', 'G#1', 'G#2', 'G#3', 'G#4', 'G#5'

In [13]:
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
note_to_int

{'0': 0,
 '0.2.5': 1,
 '0.2.6': 2,
 '0.3.6': 3,
 '0.4': 4,
 '0.4.6': 5,
 '0.4.7': 6,
 '0.5': 7,
 '1': 8,
 '1.4': 9,
 '1.4.7': 10,
 '1.4.8': 11,
 '1.5': 12,
 '1.6': 13,
 '10': 14,
 '10.0': 15,
 '10.1': 16,
 '10.1.4': 17,
 '10.2': 18,
 '10.3': 19,
 '11': 20,
 '11.0.4': 21,
 '11.2': 22,
 '11.2.5': 23,
 '11.3': 24,
 '11.3.6': 25,
 '11.3.6.7': 26,
 '11.4': 27,
 '2': 28,
 '2.4.7': 29,
 '2.4.8': 30,
 '2.5.8': 31,
 '2.5.9': 32,
 '2.6': 33,
 '2.7': 34,
 '3': 35,
 '3.5.8': 36,
 '3.5.9': 37,
 '3.6': 38,
 '3.6.10': 39,
 '3.6.9': 40,
 '3.7': 41,
 '3.8': 42,
 '4': 43,
 '4.10': 44,
 '4.6.11': 45,
 '4.6.9': 46,
 '4.7': 47,
 '4.7.11': 48,
 '4.8': 49,
 '4.8.11': 50,
 '4.9': 51,
 '5': 52,
 '5.10': 53,
 '5.9': 54,
 '6': 55,
 '6.10': 56,
 '6.10.1': 57,
 '6.11': 58,
 '6.9.0': 59,
 '7': 60,
 '7.0': 61,
 '7.10': 62,
 '7.10.0': 63,
 '7.11': 64,
 '8': 65,
 '8.1': 66,
 '8.11': 67,
 '9': 68,
 '9.0': 69,
 '9.0.4': 70,
 '9.1': 71,
 '9.11': 72,
 '9.11.3': 73,
 '9.2': 74,
 'A2': 75,
 'A3': 76,
 'A4': 77,
 'A5': 78,
 

In [14]:
network_input = []
network_output = []
print('Len ', len(notes), len(notes) - sequence_length)
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
network_input[0][:5]
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))


Len  8215 8115


[90, 91, 91, 48, 48]

In [15]:

# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [16]:
network_input[0]
network_output[0]
network_input.shape
network_output.shape

array([[ 0.6       ],
       [ 0.60666667],
       [ 0.60666667],
       [ 0.32      ],
       [ 0.32      ],
       [ 0.32      ],
       [ 0.32      ],
       [ 0.32      ],
       [ 0.32      ],
       [ 0.69333333],
       [ 0.32      ],
       [ 0.32      ],
       [ 0.60666667],
       [ 0.30666667],
       [ 0.30666667],
       [ 0.30666667],
       [ 0.30666667],
       [ 0.26666667],
       [ 0.26666667],
       [ 0.69333333],
       [ 0.26666667],
       [ 0.26666667],
       [ 0.60666667],
       [ 0.24666667],
       [ 0.24666667],
       [ 0.24666667],
       [ 0.24666667],
       [ 0.21333333],
       [ 0.21333333],
       [ 0.69333333],
       [ 0.20666667],
       [ 0.20666667],
       [ 0.60666667],
       [ 0.2       ],
       [ 0.2       ],
       [ 0.2       ],
       [ 0.2       ],
       [ 0.19333333],
       [ 0.19333333],
       [ 0.56      ],
       [ 0.06666667],
       [ 0.06666667],
       [ 0.51333333],
       [ 0.04      ],
       [ 0.04      ],
       [ 0

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])

(8115, 100, 1)

(8115, 150)

In [17]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          1574912   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
__________

In [18]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=100, batch_size=64, callbacks=callbacks_list)

Epoch 1/100
8115/8115 [==============================] - ETA: 764s - loss: 5.012 - ETA: 611s - loss: 4.987 - ETA: 554s - loss: 4.872 - ETA: 521s - loss: 4.850 - ETA: 501s - loss: 4.879 - ETA: 487s - loss: 4.842 - ETA: 477s - loss: 4.766 - ETA: 470s - loss: 4.785 - ETA: 465s - loss: 4.787 - ETA: 459s - loss: 4.729 - ETA: 454s - loss: 4.681 - ETA: 448s - loss: 4.649 - ETA: 443s - loss: 4.623 - ETA: 438s - loss: 4.583 - ETA: 434s - loss: 4.576 - ETA: 429s - loss: 4.555 - ETA: 425s - loss: 4.534 - ETA: 421s - loss: 4.523 - ETA: 417s - loss: 4.501 - ETA: 414s - loss: 4.495 - ETA: 410s - loss: 4.485 - ETA: 406s - loss: 4.474 - ETA: 402s - loss: 4.464 - ETA: 398s - loss: 4.450 - ETA: 395s - loss: 4.435 - ETA: 393s - loss: 4.424 - ETA: 390s - loss: 4.422 - ETA: 390s - loss: 4.423 - ETA: 388s - loss: 4.420 - ETA: 386s - loss: 4.414 - ETA: 386s - loss: 4.411 - ETA: 382s - loss: 4.407 - ETA: 377s - loss: 4.411 - ETA: 374s - loss: 4.406 - ETA: 370s - loss: 4.396 - ETA: 366s - loss: 4.391 - ETA: 36

8115/8115 [==============================] - ETA: 539s - loss: 3.175 - ETA: 529s - loss: 3.220 - ETA: 524s - loss: 3.485 - ETA: 519s - loss: 3.486 - ETA: 520s - loss: 3.513 - ETA: 516s - loss: 3.521 - ETA: 511s - loss: 3.459 - ETA: 506s - loss: 3.491 - ETA: 501s - loss: 3.501 - ETA: 495s - loss: 3.497 - ETA: 491s - loss: 3.492 - ETA: 487s - loss: 3.546 - ETA: 483s - loss: 3.602 - ETA: 478s - loss: 3.579 - ETA: 473s - loss: 3.580 - ETA: 469s - loss: 3.561 - ETA: 464s - loss: 3.568 - ETA: 460s - loss: 3.560 - ETA: 456s - loss: 3.562 - ETA: 451s - loss: 3.554 - ETA: 447s - loss: 3.559 - ETA: 442s - loss: 3.559 - ETA: 438s - loss: 3.552 - ETA: 434s - loss: 3.560 - ETA: 429s - loss: 3.587 - ETA: 425s - loss: 3.575 - ETA: 421s - loss: 3.569 - ETA: 417s - loss: 3.566 - ETA: 413s - loss: 3.566 - ETA: 409s - loss: 3.556 - ETA: 404s - loss: 3.545 - ETA: 400s - loss: 3.535 - ETA: 396s - loss: 3.531 - ETA: 393s - loss: 3.533 - ETA: 389s - loss: 3.531 - ETA: 384s - loss: 3.525 - ETA: 380s - loss: 3

8115/8115 [==============================] - ETA: 545s - loss: 2.702 - ETA: 534s - loss: 2.975 - ETA: 523s - loss: 2.875 - ETA: 520s - loss: 2.934 - ETA: 518s - loss: 2.982 - ETA: 514s - loss: 2.981 - ETA: 508s - loss: 2.996 - ETA: 503s - loss: 2.996 - ETA: 499s - loss: 2.998 - ETA: 494s - loss: 2.995 - ETA: 490s - loss: 2.982 - ETA: 487s - loss: 3.028 - ETA: 482s - loss: 3.036 - ETA: 478s - loss: 3.017 - ETA: 473s - loss: 3.024 - ETA: 469s - loss: 3.027 - ETA: 464s - loss: 3.021 - ETA: 459s - loss: 3.037 - ETA: 456s - loss: 3.037 - ETA: 452s - loss: 3.044 - ETA: 447s - loss: 3.028 - ETA: 443s - loss: 3.026 - ETA: 439s - loss: 3.026 - ETA: 434s - loss: 3.019 - ETA: 430s - loss: 3.008 - ETA: 426s - loss: 3.004 - ETA: 422s - loss: 3.006 - ETA: 418s - loss: 3.000 - ETA: 413s - loss: 2.989 - ETA: 409s - loss: 2.992 - ETA: 405s - loss: 2.981 - ETA: 400s - loss: 2.980 - ETA: 396s - loss: 2.972 - ETA: 392s - loss: 2.972 - ETA: 388s - loss: 2.979 - ETA: 384s - loss: 2.971 - ETA: 380s - loss: 2

8115/8115 [==============================] - ETA: 540s - loss: 2.150 - ETA: 532s - loss: 2.149 - ETA: 529s - loss: 2.244 - ETA: 522s - loss: 2.250 - ETA: 517s - loss: 2.290 - ETA: 510s - loss: 2.328 - ETA: 507s - loss: 2.340 - ETA: 503s - loss: 2.336 - ETA: 499s - loss: 2.327 - ETA: 494s - loss: 2.309 - ETA: 490s - loss: 2.301 - ETA: 488s - loss: 2.298 - ETA: 483s - loss: 2.313 - ETA: 479s - loss: 2.300 - ETA: 474s - loss: 2.294 - ETA: 470s - loss: 2.295 - ETA: 466s - loss: 2.319 - ETA: 462s - loss: 2.320 - ETA: 457s - loss: 2.343 - ETA: 453s - loss: 2.324 - ETA: 449s - loss: 2.323 - ETA: 444s - loss: 2.327 - ETA: 440s - loss: 2.326 - ETA: 436s - loss: 2.318 - ETA: 432s - loss: 2.332 - ETA: 428s - loss: 2.324 - ETA: 423s - loss: 2.329 - ETA: 419s - loss: 2.336 - ETA: 415s - loss: 2.345 - ETA: 410s - loss: 2.353 - ETA: 406s - loss: 2.354 - ETA: 402s - loss: 2.348 - ETA: 398s - loss: 2.349 - ETA: 394s - loss: 2.345 - ETA: 389s - loss: 2.344 - ETA: 385s - loss: 2.345 - ETA: 381s - loss: 2

8115/8115 [==============================] - ETA: 530s - loss: 1.710 - ETA: 529s - loss: 1.716 - ETA: 521s - loss: 1.811 - ETA: 517s - loss: 1.883 - ETA: 516s - loss: 1.899 - ETA: 512s - loss: 2.007 - ETA: 508s - loss: 1.971 - ETA: 503s - loss: 1.965 - ETA: 499s - loss: 1.946 - ETA: 496s - loss: 1.915 - ETA: 491s - loss: 1.918 - ETA: 487s - loss: 1.902 - ETA: 482s - loss: 1.911 - ETA: 478s - loss: 1.893 - ETA: 473s - loss: 1.904 - ETA: 469s - loss: 1.913 - ETA: 464s - loss: 1.919 - ETA: 459s - loss: 1.893 - ETA: 455s - loss: 1.885 - ETA: 451s - loss: 1.886 - ETA: 447s - loss: 1.885 - ETA: 442s - loss: 1.873 - ETA: 438s - loss: 1.876 - ETA: 434s - loss: 1.870 - ETA: 430s - loss: 1.853 - ETA: 425s - loss: 1.846 - ETA: 421s - loss: 1.843 - ETA: 417s - loss: 1.840 - ETA: 413s - loss: 1.837 - ETA: 408s - loss: 1.833 - ETA: 404s - loss: 1.837 - ETA: 399s - loss: 1.837 - ETA: 395s - loss: 1.832 - ETA: 391s - loss: 1.827 - ETA: 387s - loss: 1.822 - ETA: 383s - loss: 1.822 - ETA: 378s - loss: 1

KeyboardInterrupt: 

In [ ]:
model.load_weights('weights-improvement-01-4.1039-bigger.hdf5')

In [19]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]

print('init', pattern.shape, len(pattern), pattern[0])
pattern = list(pattern.reshape(-1))
prediction_output = []
# generate 500 notes
for note_index in range(50):
    print('.', end ='')
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
#     print('>', pattern.shape, len(pattern), index)
#     pattern = list(pattern.reshape(-1))
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

init (100, 1) 100 [ 0.70666667]
..................................................

In [20]:
prediction_output

['1.6',
 '6.11',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '9.1',
 '11.4',
 '11.4',
 '11.4',
 '11.4',
 '11.4',
 '11.4',
 '11.4',
 '11.4',
 '11.4',
 '11.4',
 '11.4',
 '11.4',
 '1.5',
 '1.5',
 '1.5',
 '1.5',
 '1.6',
 '1.6',
 '1.6',
 '1.6',
 '1.6',
 '1.6',
 '6.11',
 '6.11']

In [30]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Saxophone()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Saxophone()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [31]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'

In [ ]:
midi_stream.play()

In [32]:
x = midi.realtime.StreamPlayer(midi_stream)
x.play()

In [ ]:
del x